In [21]:
import stftAnSyn as stft
import dftAnSyn as dft
import SinesAnSyn as sines
import harmonicAnSyn as harms
import stochasticAnSyn as stoc
import harmResAnSyn as HR
import Utilities as U
import math

In [22]:
import numpy as np
from scipy.signal import get_window, blackmanharris, resample, blackman
from scipy.signal import chirp, triang
from scipy.fftpack import ifft, fft
import scipy.io.wavfile as wavfile
import os, sys
import matplotlib.pyplot as plt

In [ ]:
sr = 44100
f0 = 150 # Hz
f1 = 8500 # Hz
t1 = 2 # seconds
t = np.linspace(0,t1,int(t1*sr))
chirpSig = chirp(t, f0, t1, f1)
wavfile.write("chirp.wav", sr, np.int16(chirpSig*2**14))

In [ ]:
plt.plot(chirpSig[0:2000])
plt.xlabel('sample index')
plt.ylabel('sample value')

In [ ]:
(fs, x) = wavfile.read('mono_noisy_hello.wav')

In [ ]:
plt.plot(x[0:88000])
plt.xlabel('sample index')
plt.ylabel('sample value')

In [ ]:
N = 512
M = 256
H = 128
w = get_window("blackman",M)
mX, phX = stft.stftAnal(x, w, N, H)
y = stft.stftSynth(mX, phX, w.size,H)
wavfile.write("newHello.wav", fs, y)

In [ ]:
(fs, x) = wavfile.read('sine440.wav')

In [ ]:
M = 512
N = 2048
tresh = -20
win = get_window('hamming',M)
x1 = x[int(.8*fs):int(.8*fs)+M]

In [ ]:
(mX, pX) = dft.dftAnal(x1, win, N)
peaks = U.peakDetection(mX, tresh)
pMag = mX[peaks]

In [ ]:
freqaxe = fs * np.arange(N/2+1)/float(N)
plt.plot(freqaxe, mX)
plt.plot(fs * peaks/ float(N), pMag, marker='x', linestyle='')
plt.show()

In [ ]:
pLoc, pMag, pPh = U.peak_parabolicInterp(mX, pX, peaks)

In [ ]:
freqaxe = fs * np.arange(N/2+1)/float(N)
plt.plot(freqaxe, mX)
plt.plot(fs * pLoc/ float(N), pMag, marker='x', linestyle='')
plt.show()

In [ ]:
bins = np.array([-4,-3,-2,-1,0,1,2,3])
X = U.genBhLobe(bins)
plt.plot(20 * np.log10(X))

In [ ]:
sf = 44100
N = 512
peakFreq = np.array([4000])
peakMag = np.array([0])
peakPhase = np.array([0])

In [ ]:
Y = U.genSinesSpectrum(peakFreq, peakMag, peakPhase, N, sf)

In [ ]:
absY = abs(Y[:int(N/2)])
absY[absY < np.finfo(float).eps] = np.finfo(float).eps

In [ ]:
freqAxe = fs * np.arange(N/2)/float(N)
plt.plot(freqAxe, 20 * np.log10(absY))
plt.show()

In [ ]:
hN = int(N/2)
y = np.real(ifft(Y))
synthwin = np.zeros(N)
ow = triang(N/2)
synthwin[hN-H:hN+H] = ow
bh = blackmanharris(N)
bh = bh/sum(bh)
synthwin[hN-H:hN+H] = synthwin[hN-H:hN+H] / bh[hN-H:hN+H] 

In [ ]:
yw = np.zeros(N)
yw[:hN-1] = y[hN+1:]
yw[hN-1:] = y[:hN+1]
yw *= synthwin

In [ ]:
plt.plot(yw)
plt.show()

In [ ]:
(fs, x) = wavfile.read('oboe-A4.wav')
N = 512
hN = int(N/2)
H = int(N/4)
M = 511
tresh = -70
win = get_window('hamming',M)
x1 = x[int(0.8*fs):int(0.8*fs)+M]
(mX, pX) = dft.dftAnal(x1,win,N)
peaks = U.peakDetection(mX, tresh)
(peakC, peakMag, peakPh) = U.peak_parabolicInterp(mX,pX,peaks)
peakFreq = fs*peaks/float(N)
Y = U.genSinesSpectrum(peakFreq, peakMag, peakPh, N, fs)
y = np.real(ifft(Y))

In [ ]:
synthesisWindow = np.zeros(N)
ow = triang(N/2)
synthesisWindow[hN-H:hN+H] = ow
bh = blackmanharris(N)
bh = bh / sum(bh)
synthesisWindow[hN-H:hN+H] = synthesisWindow[hN-H:hN+H] / bh[hN-H:hN+H]


In [ ]:
ywin = np.zeros(N)
ywin[:hN-1] = y[hN+1:]
ywin[hN-1:] = y[:hN+1]
ywin *= synthesisWindow

In [ ]:
freqAxe = fs * np.arange(N/2)/float(N)
plt.plot(freqAxe, mX[1:])
plt.plot(fs*peaks / N, peakMag, marker='x',linestyle='')
plt.show

In [ ]:
plt.plot(ywin)
#plt.plot(y)

In [ ]:
(fs, x) = wavfile.read('VocalChop.wav')
N = 2048
M = 2000
tresh = -80
H = 128
win = get_window('hamming',M)
minDur = 0.02
maxSines = 100
freqSlope = 0.001
freqDevOff = 10

tfreq, tmag, tphase = sines.sineModelAnal(x, fs, win, N, H, tresh, maxSines, minDur, freqDevOff, freqSlope)

No = 512
out = sines.sineModelSynth(tfreq,tmag,tphase,No,H,fs)
out = np.int16(out)
wavfile.write("trackedSines.wav", fs, out)


In [ ]:
nframes = int(tfreq[:,0].size)
fT = H * np.arange(nframes)/float(fs)
tfreq[tfreq<=0] = np.nan
plt.plot(fT, tfreq)
plt.show()

In [ ]:
(fs, x) = wavfile.read('VocalChop.wav')
N = 2048
M = 1201
tresh = -90
nH = 40
win = get_window('blackman',M)
minSineDur = 0.3
minf0 = 100
maxf0 = 800
f0e = 5
harmSlope = 0.01

Ns = 512
H = 128

hfreq, hmag, hphase = harms.harmonicModelAnal(x, fs, win, N, H, tresh, nH, minf0, maxf0, f0e, harmSlope, minSineDur)

numFrames = int(hfreq[:,0].size)
fTime = H*np.arange(numFrames)/float(fs)
hfreq[hfreq<=0] = np.nan

plt.plot(fTime, hfreq)
plt.show()

out = harms.harmonicModel(x, fs, win, N, tresh, nH, minf0, maxf0, f0e)
out = np.int16(out)
wavfile.write("trackedHarmonics.wav", fs, out)


In [ ]:
(fs, x) = wavfile.read('seashore-waves-mono.wav')

H = 256
N = 512
sf = 0.1

env = stoc.stochasticAnal(x, H, N, sf)

out = stoc.stochasticSynth(env,H,N)
out = np.int16(out)
wavfile.write("stochastic-wave.wav", fs, out)

plt.plot(np.arange(out.size)/float(fs),out)

In [23]:
(fs,x) = wavfile.read('VocalChop.wav')
begin = 40000
M = 801
N = 2048
t = -100
minf0 = 300
maxf0 = 500
f0e = 5
nH = 60
harmSl = .01

w = get_window('blackman', M)
hM1 = int(math.floor((M+1)/2))
hM2 = int(math.floor(M/2))

x1 = x[begin-hM1:begin+hM2]
mX, pX = dft.dftAnal(x1, w, N)
ploc = U.peakDetection(mX, t)
iploc, pmag, ppx = U.peak_parabolicInterp(mX, pX, ploc)
ipf = fs*iploc/N
f0 = U.f0Twm(ipf, pmag, f0e, minf0, maxf0,0)
hfr, hmag, hpx = harms.harmonicDetection(ipf,pmag,ppx, f0, nH, [], fs, harmSl)

Ns = 512
hNs = 256
Yh = U.genSinesSpectrum(hfr, hmag, hpx, Ns, fs)

wr = get_window('blackmanharris', Ns)
xw2 = x[begin-hNs-1:begin+hNs-1] * wr / sum(wr)
fftbuffer = np.zeros(Ns)
fftbuffer[:hNs] = xw2[hNs:]
fftbuffer[hNs:] = xw2[:hNs]
x2 = fft(fftbuffer)
Xr = x2 - Yh

In [24]:
N = 1024
M = 601 
Ns = 512
H = 128
minSineDur = 0.1
nH = 100

hfreq, hmag, hphase, xr = HR.hprModelAnal(x, fs, w, N, H, t, minSineDur, nH, minf0, maxf0, f0e, harmSl)
mXr, pXr = stft.stftAnal(xr, w, N, H)
y, yh = HR.hprModelSynth(hfreq, hmag, hphase, xr, Ns, H, fs)

out1 = np.int16(yh)
out2 = np.int16(xr)
out3 = np.int16(y)
wavfile.write("hr_sines.wav", fs, out1)
wavfile.write("hr_residual.wav", fs, out2)
wavfile.write("hr_output.wav", fs, out3)